<a href="https://colab.research.google.com/github/arthurdepina/Check-for-Check/blob/main/runningMPI_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving rand.txt to rand.txt


{'rand.txt': b'41\n67\n34\n0\n69\n24\n78\n58\n62\n64\n5\n45\n81\n27\n61\n91\n95\n42\n27\n36\n91\n4\n2\n53\n92\n82\n21\n16\n18\n95\n47\n26\n71\n38\n69\n12\n67\n99\n35\n94\n3\n11\n22\n33\n73\n64\n41\n11\n53\n68\n47\n44\n62\n57\n37\n59\n23\n41\n29\n78\n16\n35\n90\n42\n88\n6\n40\n42\n64\n48\n46\n5\n90\n29\n70\n50\n6\n1\n93\n48\n29\n23\n84\n54\n56\n40\n66\n76\n31\n8\n44\n39\n26\n23\n37\n38\n18\n82\n29\n41\n33\n15\n39\n58\n4\n30\n77\n6\n73\n86\n21\n45\n24\n72\n70\n29\n77\n73\n97\n12\n86\n90\n61\n36\n55\n67\n55\n74\n31\n52\n50\n50\n41\n24\n66\n30\n7\n91\n7\n37\n57\n87\n53\n83\n45\n9\n9\n58\n21\n88\n22\n46\n6\n30\n13\n68\n0\n91\n62\n55\n10\n59\n24\n37\n48\n83\n95\n41\n2\n50\n91\n36\n74\n20\n96\n21\n48\n99\n68\n84\n81\n34\n53\n99\n18\n38\n0\n88\n27\n67\n28\n93\n48\n83\n7\n21\n10\n17\n13\n14\n9\n16\n35\n51\n0\n49\n19\n56\n98\n3\n24\n8\n44\n9\n89\n2\n95\n85\n93\n43\n23\n87\n14\n3\n48\n0\n58\n18\n80\n96\n98\n81\n89\n98\n9\n57\n72\n22\n38\n92\n38\n79\n90\n57\n58\n91\n15\n88\n56\n11\n2\n34\n72\n55\n

In [2]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-bin set to manually installed.
openmpi-common is already the newest version (4.1.2-2ubuntu1).
openmpi-common set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [9]:
%%writefile teste-mpi.c
#include "mpi.h"
#include <stdio.h>
#include <math.h>
#include <unistd.h>
#include <stdlib.h>

#define MAXSIZE 1000
int main(int argc, char **argv)
{
	int myid, numprocs;
	int data[MAXSIZE], i, x, low, high, myresult=0, result=0;
	FILE *fp;
	MPI_Init(&argc,&argv);
	MPI_Comm_size(MPI_COMM_WORLD,&numprocs);
	MPI_Comm_rank(MPI_COMM_WORLD,&myid);
	if (myid == 0) {  /* Open input file and initialize data */
		if ((fp = fopen("rand.txt","r")) == NULL) {
			printf("Can't open the input file: rand.txt\n\n");
			exit(1);
		}
		for(i = 0; i < MAXSIZE; i++) fscanf(fp,"%d\n", &data[i]);
	}
	MPI_Bcast(data, MAXSIZE, MPI_INT, 0, MPI_COMM_WORLD); /* broadcast data */
	x = MAXSIZE/numprocs; /* Add my portion Of data */
	low = myid * x;
	high = low + x;
	for(i = low; i < high; i++)
		myresult += data[i];
	char hostname[30];
	gethostname(hostname, 30);
	printf("I got %d from %d - %s\n", myresult, myid, hostname); /* Compute global sum */
	MPI_Reduce(&myresult, &result, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
	if (myid == 0) printf("The sum is %d.\n", result);
	MPI_Finalize();
	return 0;
}



Overwriting teste-mpi.c


In [10]:
!mpicc -o teste-mpi teste-mpi.c

In [11]:
!mpiexec --allow-run-as-root --oversubscribe -n 4 ./teste-mpi > output.txt
!cat output.txt

I got 11968 from 2 - e6bccd87dc76
I got 11689 from 3 - e6bccd87dc76
I got 12677 from 1 - e6bccd87dc76
I got 11921 from 0 - e6bccd87dc76
The sum is 48255.


In [20]:
%%writefile mpi_pi_scatter_gather.c
// calculo pi scatter e gather

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    long long n = 1000000;
    if (argc > 1) {
        n = atoll(argv[1]);
    }

    int *work_info = NULL;
    if (rank == 0) {
        work_info = (int*) malloc(size * 2 * sizeof(int));
        int base = n / size;
        int rem = n % size;
        int current_start = 0;
        for (int i = 0; i < size; i++) {
            int count = base + (i < rem ? 1 : 0);
            work_info[2 * i]     = current_start;
            work_info[2 * i + 1] = count;
            current_start += count;
        }
    }

    int local_info[2];
    MPI_Scatter(work_info, 2, MPI_INT, local_info, 2, MPI_INT, 0, MPI_COMM_WORLD);
    if (rank == 0) {
        free(work_info);
    }
    int start = local_info[0];
    int count = local_info[1];


    double local_sum = 0.0;
    int sign = (start % 2 == 0) ? 1 : -1;
    for (int i = 0; i < count; i++) {
        int index = start + i;
        local_sum += sign / (double)(2 * index + 1);
        sign = -sign;
    }

    double *all_sums = NULL;
    if (rank == 0) {
        all_sums = (double*) malloc(size * sizeof(double));
    }
    MPI_Gather(&local_sum, 1, MPI_DOUBLE, all_sums, 1, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        double global_sum = 0.0;
        for (int i = 0; i < size; i++) {
            global_sum += all_sums[i];
        }
        double pi = 4.0 * global_sum;
        printf("Pi (scatter/gather): %.15f\n", pi);
        free(all_sums);
    }

    MPI_Finalize();
    return 0;
}

Overwriting mpi_pi_scatter_gather.c


In [21]:
!mpicc -o mpi_pi_scatter_gather mpi_pi_scatter_gather.c
!mpiexec --allow-run-as-root --oversubscribe -n 4 ./mpi_pi_scatter_gather

Pi (scatter/gather): 3.141591653589781


In [18]:
%%writefile mpi_pi_send_receive.c
// calculo pi send e receive

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    long long n = 1000000;
    if (argc > 1) {
        n = atoll(argv[1]);
    }

    int start, count;
    if (rank == 0) {
        int base = n / size;
        int rem = n % size;
        int current_start = 0;
        count = base + (0 < rem ? 1 : 0);
        start = current_start;
        current_start += count;
        for (int i = 1; i < size; i++) {
            int temp_count = base + (i < rem ? 1 : 0);
            int temp_start = current_start;
            current_start += temp_count;
            int work[2] = {temp_start, temp_count};
            MPI_Send(work, 2, MPI_INT, i, 0, MPI_COMM_WORLD);
        }
    } else {
        int work[2];
        MPI_Recv(work, 2, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        start = work[0];
        count = work[1];
    }

    double local_sum = 0.0;
    int sign = (start % 2 == 0) ? 1 : -1;
    for (int i = 0; i < count; i++) {
        int index = start + i;
        local_sum += sign / (double)(2 * index + 1);
        sign = -sign;
    }

    if (rank == 0) {
        double global_sum = local_sum;
        double temp_sum;
        for (int i = 1; i < size; i++) {
            MPI_Recv(&temp_sum, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            global_sum += temp_sum;
        }
        double pi = 4.0 * global_sum;
        printf("Pi (send/receive): %.15f\n", pi);
    } else {
        MPI_Send(&local_sum, 1, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}

Overwriting mpi_pi_send_receive.c


In [19]:
!mpicc -o mpi_pi_send_receive mpi_pi_send_receive.c
!mpiexec --allow-run-as-root --oversubscribe -n 4 ./mpi_pi_send_receive

Pi (send/receive): 3.141591653589781
